In [32]:
import pandas as pd
import numpy as np
import json
import tensorflow as tf
import tensorflow_hub as hub
from keras.models import load_model

In [49]:
f = open('./report-response-IPW6XqsRKz1trDdqrzNnD.json')
deserialized_event = json.load(f)

In [52]:
data = deserialized_event['data']
  
total_request_numpy = []

for i in range(len(data)):
    key_column = []
    value_coulmn = []
    for j in range(len(data[i]['req'])):
      key_column.append(data[i]['req'][j][0])
      value_coulmn.append(data[i]['req'][j][1])
    kv_column = np.array(data[i]['req'])
    url_column = np.full((kv_column.shape[0], 1), data[i]['url'])
    timestamp_column = np.full((kv_column.shape[0], 1), data[i]['timeStamp'])
    request_numpy = np.column_stack((key_column, value_coulmn, url_column, timestamp_column))
    total_request_numpy.append(request_numpy)

total = []
for req in total_request_numpy:
    for row in req:
        total.append(row)

df = pd.DataFrame(total, columns=['key', 'value', 'url', 'timestamp'])
        
keys = df['key'].tolist()
df

C:\Users\mattr\AppData\Local\Temp\ipykernel_22740\2750371883.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  kv_column = np.array(data[i]['req'])
c:\Users\mattr\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\shape_base.py:652: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(v)


,key,value,url,timestamp
0,0,O43z0dpjhgX20SCx4KAo,https://jnn-pa.googleapis.com/$rpc/google.inte...,1682636558733.325
1,context.adSignalsInfo.params[0].key,dt,https://www.youtube.com/youtubei/v1/att/get?ke...,1682636558775.455078
2,context.adSignalsInfo.params[0].value,1682636557099,https://www.youtube.com/youtubei/v1/att/get?ke...,1682636558775.455078
3,context.adSignalsInfo.params[1].key,flash,https://www.youtube.com/youtubei/v1/att/get?ke...,1682636558775.455078
4,context.adSignalsInfo.params[1].value,0,https://www.youtube.com/youtubei/v1/att/get?ke...,1682636558775.455078
...,...,...,...,...
647,1.variables.channelID,557148694,https://gql.twitch.tv/gql#origin=twilight,1682636567140.09
648,2.extensions.persistedQuery.sha256Hash,870906a2de25d7488239dbeb947dafe3e5697f1fef2e8b...,https://gql.twitch.tv/gql#origin=twilight,1682636567140.09
649,2.extensions.persistedQuery.version,1,https://gql.twitch.tv/gql#origin=twilight,1682636567140.09
650,2.operationName,FollowButton_User,https://gql.twitch.tv/gql#origin=twilight,1682636567140.09


In [35]:
def split_word(word):
  s = word.replace(".", " ").replace("-", " ")
  for i in range(len(s) - 1):
    if s[i].islower() and s[i + 1].isupper():
      s = s[0:i+1] + " " + s[i + 1:]
  
  return s.lower()

In [36]:
keys = map(split_word, keys)
keys = list(keys)
keys

['operation name',
 'query',
 'variables is live',
 'variables is vod',
 'variables login',
 'variables player type',
 'variables vod id',
 '0 extensions persisted query sha256hash',
 '0 extensions persisted query version',
 '0 operation name',
 '0 variables include private',
 '0 variables vod id',
 '1 extensions persisted query sha256hash',
 '1 extensions persisted query version',
 '1 operation name',
 '1 variables include private',
 '1 variables video id',
 '2 extensions persisted query sha256hash',
 '2 extensions persisted query version',
 '2 operation name',
 '2 variables input owns video id',
 '3 extensions persisted query sha256hash',
 '3 extensions persisted query version',
 '3 operation name',
 '3 variables has video id',
 '3 variables video id']

In [37]:
#load and embed using USE
module = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

embedded_keys = module(keys)
embedded_keys

<tf.Tensor: shape=(26, 512), dtype=float32, numpy=
array([[-0.03038407,  0.02269426, -0.03270616, ..., -0.05274822,
         0.0301472 , -0.03571258],
       [-0.05990529, -0.04157299,  0.06171313, ..., -0.03685904,
         0.02025132, -0.04836494],
       [ 0.01832912, -0.06247625, -0.08174341, ...,  0.00878124,
         0.03856565, -0.01781394],
       ...,
       [-0.04853867, -0.01225588, -0.00620967, ..., -0.04599472,
         0.01915557, -0.01555176],
       [ 0.01596977, -0.0535123 , -0.01530978, ..., -0.02261168,
        -0.01057775, -0.00992327],
       [ 0.02391768, -0.03327496, -0.02885165, ..., -0.03248364,
        -0.010353  ,  0.01494709]], dtype=float32)>

In [38]:
#load multiclassifier
model = load_model('./privacy_text_classifier.h5')

In [39]:
def get_class(prediction):
  i = 0
  c = 0
  m = 0
  for pred in prediction:
    if pred > m:
        m = pred
        c = i
    i += 1
  return c

In [40]:
predictions = model.predict(embedded_keys)

print(f"there are {len(predictions)} predictions")

print(predictions[0])

1/1 [==============================] - 1s 1s/step
there are 26 predictions
[0.6234514  0.022054   0.13053253 0.10583443 0.11812761]


In [41]:
classes = []
for p in predictions:
   classes.append(get_class(p))

classes

[0, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 0, 4, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4]

In [46]:
kv_dict = []
for index, row in df.iterrows():
    kv_dict.append(
        {
            'key' : row['key'],
            'value' : row['value'],
            'prediction' : classes[index],
            'url' : row['url'],
            'timestamp' : row['timestamp']
        }
    )